## ひろしまQuest2020#stayhome【球種予測部門】
#### all_merge_14
- 左右で球種を別に集計

#### all_merge_13
- 前の投球からの時間差

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
import gc
%matplotlib inline
pd.set_option('display.max_Columns', 100)

In [2]:
ALL_MERGE = 'intermediate/all_merge_14.f'
SUBMIT = 'submit/ball_14.csv'

In [3]:
all_pitch = pd.read_feather(ALL_MERGE)
all_pitch.shape

(778767, 168)

### 不要な列を削除

In [4]:
all_pitch.drop(
    columns=[
        '年度', 
        '試合ID', 
        'ホームチームID', 'アウェイチームID', 
        '投手ID', '投手チームID', 
        '打者ID', '打者チームID', 
        'プレイ前走者状況', 
        '捕手ID', 
        'opening_date', 'game_date',
        'start_time', 'game_time', 'elapsed_time'
    ], inplace=True)

### Rename

In [5]:
all_pitch.rename(columns={
    'データ内連番': 'No',
    '投球位置区域': 'pitch_area',
    '試合内連番': 'No_in_game',
    '試合内投球数': 'pitch_cnt_in_game',
    'イニング': 'inning',
    'イニング内打席数': 'bat_cnt_in_inning',
    '打席内投球数': 'pitch_cnt_in_bat',
    '投手登板順': 'pitch_order',
    '投手試合内対戦打者数': 'player_cnt_in_game',
    '投手試合内投球数': 'pitcher_cnt_in_game',
    '投手イニング内投球数': 'pitcher_cnt_in_inning',
    '打者打順': 'bat_order',
    '打者試合内打席数': 'bat_cnt_in_game',
    'プレイ前ホームチーム得点数': 'home_point',
    'プレイ前アウェイチーム得点数': 'away_point',
    'プレイ前アウト数': 'out_cnt',
    'プレイ前ボール数': 'ball_cnt',
    'プレイ前ストライク数': 'strike_cnt',
}, inplace=True)

In [6]:
print(all_pitch.shape)
column_cnt = len(all_pitch.columns)
all_pitch.head()

(778767, 153)


,No,ball,pitch_area,No_in_game,pitch_cnt_in_game,inning,bat_cnt_in_inning,pitch_cnt_in_bat,pitch_order,player_cnt_in_game,pitcher_cnt_in_game,pitcher_cnt_in_inning,bat_order,bat_cnt_in_game,home_point,away_point,out_cnt,ball_cnt,strike_cnt,bc_straight,bc_curve,bc_slider,bc_shoot,bc_fork,bc_changeup,bc_sinker,bc_cutball,bc_area0,bc_area1,bc_area2,bc_area3,bc_area4,bc_area5,bc_area6,bc_area7,bc_area8,bc_area9,bc_area10,bc_area11,bc_area12,first,second,third,base_cnt,pitch_LR,bat_LR,top_bot,role,pos_pit,date_from_opening,...,weight_bat,draft_order_bat,salary_bat,foreigner_bat,company_bat,univ_bat,highsch_bat,age_bat,play_year_bat,salary_year_bat,salary_x_year_bat,bmi_bat,batter_cnt,bat_game_cnt,firm,height,weight,draft_order,salary,foreigner,company,univ,highsch,age,play_year,salary_year,salary_x_year,bmi,salary_dif_p-b,play_year_dif_p-b,age_dif_p-b,salary_year_dif_p-b,salary_x_year_dif_p-b,bmi_dif_p-b,salary_dif_p-c,play_year_dif_p-c,age_dif_p-c,salary_year_dif_p-c,salary_x_year_dif_p-c,bmi_dif_p-c,salary_dif_b-c,play_year_dif_b-c,age_dif_b-c,salary_year_dif_b-c,salary_x_year_dif_b-c,bmi_dif_b-c,pit_bat_L_L,pit_bat_L_R,pit_bat_R_L,pit_bat_R_R
0,0,0.0,0.0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0.477023,0.097592,0.200870,0.074454,0.046799,0.041882,0.008619,0.052760,0.048489,0.071105,0.059259,0.047675,0.063792,0.049150,0.046338,0.070075,0.059013,0.117934,0.090325,0.114800,0.162045,0,0,0,0,0.0,1.0,1,1,0,0,...,74,5.0,15000,0,1,1,0,32,8.0,1875.0,120000.0,23.889463,523.0,119.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,2.141358,19000,2.0,1,2750.0,124000.0,-0.165367,10000,4.0,4,625.0,100000.0,-2.306725,0,0,1,0
1,1,0.0,8.0,2,2,1,1,2,1,1,2,2,1,1,0,0,0,0,1,0.459832,0.090366,0.161893,0.068313,0.099045,0.061079,0.012505,0.046967,0.035008,0.056450,0.047706,0.035812,0.044074,0.034687,0.037484,0.058379,0.050407,0.112451,0.103867,0.161700,0.221976,0,0,0,0,0.0,1.0,1,1,0,0,...,74,5.0,15000,0,1,1,0,32,8.0,1875.0,120000.0,23.889463,523.0,119.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,2.141358,19000,2.0,1,2750.0,124000.0,-0.165367,10000,4.0,4,625.0,100000.0,-2.306725,0,0,1,0
2,2,0.0,5.0,3,3,1,1,3,1,1,3,3,1,1,0,0,0,0,2,0.469730,0.066062,0.159395,0.031361,0.162940,0.062722,0.014112,0.033679,0.021884,0.035656,0.030884,0.020316,0.021748,0.021816,0.028429,0.036678,0.029520,0.133761,0.131170,0.201595,0.286542,0,0,0,0,0.0,1.0,1,1,0,0,...,74,5.0,15000,0,1,1,0,32,8.0,1875.0,120000.0,23.889463,523.0,119.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,2.141358,19000,2.0,1,2750.0,124000.0,-0.165367,10000,4.0,4,625.0,100000.0,-2.306725,0,0,1,0
3,3,0.0,12.0,4,4,1,2,1,1,2,4,4,2,1,0,0,1,0,0,0.477023,0.097592,0.200870,0.074454,0.046799,0.041882,0.008619,0.052760,0.048489,0.071105,0.059259,0.047675,0.063792,0.049150,0.046338,0.070075,0.059013,0.117934,0.090325,0.114800,0.162045,0,0,0,0,0.0,0.0,1,1,0,0,...,74,1.0,7040,0,0,0,1,40,22.0,320.0,154880.0,22.839506,268.0,81.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,1250.0,20000.0,26.196187,16960,-16.0,-11,3680.0,-10880.0,3.191314,19000,2.0,1,2750.0,124000.0,-0.165367,2040,18.0,12,-930.0,134880.0,-3.356681,0,0,0,1
4,4,0.0,8.0,5,5,1,2,2,1,2,5,5,2,1,0,0,1,0,1,0.459832,0.090366,0.161893,0.068313,0.099045,0.061079,0.012505,0.046967,0.035008,0.056450,0.047706,0.035812,0.044074,0.034687,0.037484,0.058379,0.050407,0.112451,0.103867,0.161700,0.221976,0,0,0,0,0.0,0.0,1,1,0,0,...,74,1.0,7040,0,0,0,1,40,22.0,320.0,154880.0,22.839506,268.0,81.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,1250.0,20000.0,26.196187,16960,-16.0,-11,3680.0,-10880.0,3.191314,19000,2.0,1,2750.0,124000.0,-0.165367,2040,18.0,12,-930.0,134880.0,-3.356681,0,0,0,1


### train
- 行数: 257117

In [7]:
train = all_pitch.dropna(subset=['pitch_area'])
train.shape

(257117, 153)

### test
- 行数: 521650

In [8]:
test = all_pitch[all_pitch['pitch_area'].isnull()]
test.shape

(521650, 153)

In [9]:
del all_pitch
gc.collect()

60

In [10]:
train_d = train.drop([
    'No', 
    'pitch_area', 
    'ball'
], axis=1)

In [11]:
test_d = test.drop([
    'No', 
    'pitch_area', 
    'ball'
], axis=1)

## Dataset作成

In [12]:
lgb_train = lgb.Dataset(train_d, train['ball'])

### LGBM学習
- multiclass : クラス数=8
- tuning結果のパラメータを使用
- 'learning_rate' : 0.1

In [13]:
%%time
lgb_param = {
        'objective' : 'multiclass',
        'boosting_type': 'gbdt',
        'metric' : 'multi_logloss',
        'num_class' : 8,
        'seed' : 0,
        'learning_rate' : 0.1,
        'lambda_l1': 6.9923570049658075, 
        'lambda_l2': 0.002378623984798833, 
        'num_leaves': 18, 
        'feature_fraction': 0.45199999999999996, 
        'bagging_fraction': 0.9799724836460725, 
        'bagging_freq': 4, 
        'min_child_samples': 20
    }

# 学習
cv_results = lgb.cv(lgb_param, lgb_train,
                    num_boost_round=15000,
                    early_stopping_rounds=100,
                    verbose_eval=100,
                    nfold=4)

num_boost_round = len(cv_results['multi_logloss-mean'])
print('Best num_boost_round:', num_boost_round)
best_cv_score = cv_results['multi_logloss-mean'][-1]
print('Best CV score:', best_cv_score)

[100]	cv_agg's multi_logloss: 1.1428 + 0.00224384
[200]	cv_agg's multi_logloss: 1.12187 + 0.00245945
[300]	cv_agg's multi_logloss: 1.11501 + 0.00257504
[400]	cv_agg's multi_logloss: 1.11093 + 0.00266061
[500]	cv_agg's multi_logloss: 1.1081 + 0.00274319
[600]	cv_agg's multi_logloss: 1.10616 + 0.00292669
[700]	cv_agg's multi_logloss: 1.10467 + 0.00300669
[800]	cv_agg's multi_logloss: 1.10373 + 0.00302347
[900]	cv_agg's multi_logloss: 1.10287 + 0.00316824
[1000]	cv_agg's multi_logloss: 1.1025 + 0.00313861
[1100]	cv_agg's multi_logloss: 1.10229 + 0.00313148
[1200]	cv_agg's multi_logloss: 1.10222 + 0.00323108
[1300]	cv_agg's multi_logloss: 1.10222 + 0.00334984
Best num_boost_round: 1240
Best CV score: 1.1021952294615331
CPU times: user 1h 32min 7s, sys: 17.4 s, total: 1h 32min 24s
Wall time: 11min 34s


In [14]:
best_iter = int(num_boost_round * 1.1)

### 全体で再学習

In [15]:
%%time
lgb_model = lgb.train(lgb_param, lgb_train, num_boost_round=best_iter)

CPU times: user 28min 38s, sys: 5.26 s, total: 28min 43s
Wall time: 3min 35s


### Feature Importance

In [16]:
fi = lgb_model.feature_importance()
fn = lgb_model.feature_name()
df_feature_importance = pd.DataFrame({'feat_name':fn, 'feat_imp':fi})
df_feature_importance.sort_values('feat_imp', inplace=True)
df_feature_importance.tail(30)

,feat_name,feat_imp
94,total,2094
91,changeup,2233
99,height_bat,2260
100,weight_bat,2331
89,shoot,2367
6,player_cnt_in_game,2408
132,salary_x_year_dif_p-b,2418
128,salary_dif_p-b,2423
144,salary_x_year_dif_b-c,2508
140,salary_dif_b-c,2518


In [17]:
%%time
predict = lgb_model.predict(test_d, num_iteration = lgb_model.best_iteration)

CPU times: user 7min 3s, sys: 656 ms, total: 7min 4s
Wall time: 54.2 s


## 結果出力

In [18]:
submit = pd.DataFrame(predict)
submit.reset_index(inplace=True)
print(submit.shape)
submit.head()

(521650, 9)


,index,0,1,2,3,4,5,6,7
0,0,0.498911,0.028753,0.119000,0.269152,0.002233,0.000013,0.000009,0.081930
1,1,0.534715,0.064428,0.088838,0.282986,0.003204,0.000008,0.000004,0.025816
2,2,0.379324,0.078461,0.123474,0.252427,0.106101,0.000031,0.000025,0.060157
3,3,0.374209,0.081980,0.166983,0.319945,0.018223,0.000029,0.000007,0.038624
4,4,0.234000,0.046326,0.307718,0.309082,0.014216,0.000031,0.000010,0.088616


### 2017の平均
- ストレート           0.468254
- カーブ             0.074725
- スライダー           0.185806
- シュート            0.070633
- フォーク            0.083013
- チェンジアップ         0.054217
- シンカー            0.011361
- カットボール          0.051992

In [19]:
submit.mean()

index    260824.500000
0             0.522052
1             0.071389
2             0.182132
3             0.054906
4             0.070726
5             0.048322
6             0.007275
7             0.043199
dtype: float64

In [20]:
submit.to_csv(SUBMIT, header=False, index=False)

### SIGNATE

In [21]:
'signate submit --competition-id=275 ~/work/baseball/{} --note feat={}_cv={}'.format(SUBMIT, column_cnt, best_cv_score)

'signate submit --competition-id=275 ~/work/baseball/submit/ball_14.csv --note feat=153_cv=1.1021952294615331'

### GCS

In [22]:
'gsutil cp ~/work/baseball/{} gs://yasunobu_baseball/submit/'.format(SUBMIT)

'gsutil cp ~/work/baseball/submit/ball_14.csv gs://yasunobu_baseball/submit/'

### 評価結果
#### 14 (153) ... 左右で球種を別に集計
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1240
- Best CV score: 1.1021952294615331
- Wall time: 11min 34s
- 評価結果  : 1.3981900

#### 13 (157) ... 前の投球からの時間差を追加
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1416
- Best CV score: 1.1062463941695753
- Wall time: 13min 10s
- 評価結果  : 1.4012184

#### 12 (156) ... コースの集計を追加
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1353
- Best CV score: 1.106942086205103
- Wall time: 27min 41s
- 評価結果  : 1.4016077

#### 11 (143) ... ボールカウントの集計を追加
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1209
- Best CV score: 1.1075153721728936
- Wall time: 23min 19s
- 評価結果  : 未提出

#### 10_tune (135) ... 再チューニング
- 'num_leaves' : 15,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1260
- Best CV score: 1.1118897437428692
- Wall time: 8min 40s
- 評価結果  : 1.4060397

#### 10s (135) ... trainを75%サンプリング
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1412
- Best CV score: 1.1190287120710383
- Wall time: 12min 5s
- 評価結果  : 1.4310869

#### 10 (135) ... 5相当に戻した
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1112
- Best CV score: 1.111265000190272
- Wall time: 9min 30s
- 評価結果  : 1.4008712

#### 9 (136) ... 5相当に戻した=>NG
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1106
- Best CV score: 1.1113177029272225
- Wall time: 25min 44s (local)
- 評価結果  : 1.5133521

#### 8 (157)
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1100
- Best CV score: 1.1093728644236824
- Wall time: 10min 4s
- 評価結果  : 1.4925832

#### 7 (235)
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1204
- Best CV score: 1.108590082437519
- Wall time: 12min 56s
- 評価結果  : 1.4946937

#### 6 (207)
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1135
- Best CV score: 1.1086837935965734
- Wall time: 11min 54s
- 評価結果  : 1.3999194

#### 5-tune (参考)
- 'num_leaves' : 18,
- 'learning_rate' : 0.05,
- Best num_boost_round: 2395
- Best CV score: 1.1103399384119979
- Wall time: 18min 29s (n1-standard-8)

#### 5-tune (135)
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- CV score: 1.1112402566616455
- 評価結果  : 1.3991780
- CV Wall time: 27min 48s (local) -> 18min 24s (n1-standard-4) -> 9min 3s (n1-standard-8)

#### 5 (135)
- 'num_leaves' : 31,
- 'learning_rate' : 0.1,
- CV score: 1.1161825304353852
- 評価結果  : 1.4252286

#### 4 (128)
- CV score: 1.1158111693845418
- 評価結果  : 1.4299265

#### 3 (97)
- CV score: 1.1172290313076036
- 評価結果  : 1.4335806